## **1 Tiny ImageNet Challenge**

In Assignment 4, you trained an image classifier using CIFAR-10 dataset, which does not require sophisti-
cated data preprocessing or model design choice to achieve a decent accuracy. In this section, your task is to train a classifier for a more difficult dataset – Tiny ImageNet. Please use the URL below to access the Kaggle page of this competition.

https://www.kaggle.com/t/3104a72abd7c4d019923cb870855b732

The Kaggle page is for evaluation and submission only, please refer to this instructions for details.

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import PReLU
from keras.initializers import Constant

from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
   
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau


In [3]:

import time

import PIL.Image as Image

import tensorflow_hub as hub

import datetime
from glob import glob
%load_ext tensorboard

In [4]:
!wget https://storage.googleapis.com/cse144/tiny-imagenet-100.zip
!unzip 'tiny-imagenet-100.zip'

--2022-03-16 05:14:53--  https://storage.googleapis.com/cse144/tiny-imagenet-100.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113846330 (109M) [application/zip]
Saving to: ‘tiny-imagenet-100.zip.1’

tiny-imagenet-100.z 100%[===================>] 108.57M   280MB/s    in 0.4s    

2022-03-16 05:14:53 (280 MB/s) - ‘tiny-imagenet-100.zip.1’ saved [113846330/113846330]

Archive:  tiny-imagenet-100.zip
replace tiny-imagenet-100/submission_sample.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
train_dir = glob('/content/tiny-imagenet-100/train/*/*/*')
test_dir = glob('/content/tiny-imagenet-100/test/*/*')

In [6]:
label_dict = {}
list_of_wnids = [0] * 100
for i, line in enumerate(open("tiny-imagenet-100/wnids.txt", "r")): 
    label_dict[line.rstrip("\n")] = i
    list_of_wnids[i] = line.strip('\n')
    
print(label_dict)
print(list_of_wnids)

{'n02124075': 0, 'n04067472': 1, 'n04540053': 2, 'n04099969': 3, 'n07749582': 4, 'n01641577': 5, 'n02802426': 6, 'n09246464': 7, 'n07920052': 8, 'n03970156': 9, 'n03891332': 10, 'n02106662': 11, 'n03201208': 12, 'n02279972': 13, 'n02132136': 14, 'n04146614': 15, 'n07873807': 16, 'n02364673': 17, 'n04507155': 18, 'n03854065': 19, 'n03838899': 20, 'n03733131': 21, 'n01443537': 22, 'n07875152': 23, 'n03544143': 24, 'n09428293': 25, 'n03085013': 26, 'n02437312': 27, 'n07614500': 28, 'n03804744': 29, 'n04265275': 30, 'n02963159': 31, 'n02486410': 32, 'n01944390': 33, 'n09256479': 34, 'n02058221': 35, 'n04275548': 36, 'n02321529': 37, 'n02769748': 38, 'n02099712': 39, 'n07695742': 40, 'n02056570': 41, 'n02281406': 42, 'n01774750': 43, 'n02509815': 44, 'n03983396': 45, 'n07753592': 46, 'n04254777': 47, 'n02233338': 48, 'n04008634': 49, 'n02823428': 50, 'n02236044': 51, 'n03393912': 52, 'n07583066': 53, 'n04074963': 54, 'n01629819': 55, 'n09332890': 56, 'n02481823': 57, 'n03902125': 58, 'n0340

In [7]:
X = []
y = []
for i in train_dir:
    for key in label_dict.keys():
        if key == i[50:59]:
            y.append(np.array([label_dict.get(key)]))
    img = Image.open(i).convert("RGB")

    X.append(np.array(img))
    #X.append(np.array(Image.open(i).convert("RGB")))

x_training = np.asarray(X)
y_training = np.asarray(y)
print(x_training.shape)
print(y_training.shape)

(50000, 64, 64, 3)
(50000, 1)


In [8]:
x_train, x_val = train_test_split(
    x_training, test_size=0.1, random_state=144, shuffle=True
)
y_train, y_val = train_test_split(
    y_training, test_size=0.1, random_state=144, shuffle=True
)

In [9]:
from tensorflow.keras.layers import Input
#from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

In [10]:
# Import the VGG16 trained neural network model, minus it's last (top) neuron layer.
base_model = ResNet50(weights = 'imagenet', 
                   include_top = False, 
                   input_shape = (64, 64, 3), 
                   pooling = None)

for i in base_model.layers[0:50]:
    i.trainable = False
for i in base_model.layers[0:-80]:
    i.trainable = False

94781440/94765736 [==============================] - 1s 0us/step


In [11]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [12]:
# Now add layers to our pre-trained base model and add classification layers on top of it
x = base_model.output
x = normalization_layer(x)
x = layers.BatchNormalization()(x)
x = Flatten()(x)
x = layers.Dropout(0.5)(x)
x = Dense(1024, activation = 'relu')(x)
#x = layers.Dropout(0.5)(x)
predic = Dense(100, activation = 'softmax')(x) 

# And now put this all together to create our new model.
model = Model(inputs = base_model.input, outputs = predic) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [ ]:
model.compile(optimizer = Adagrad(learning_rate = 0.01),
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])

In [13]:
model.compile(
  #optimizer=tf.keras.optimizers.Adam(),
  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9),
  loss = 'sparse_categorical_crossentropy', 
  #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  #loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['acc'])


Train the Model

In [14]:
NUM_EPOCHS = 15
batch_size = 128

history = model.fit(x_train, y_train,
                    epochs=NUM_EPOCHS,
                    batch_size=batch_size,
                    validation_data=(x_val,y_val))

Epoch 1/15
352/352 [==============================] - 46s 87ms/step - loss: 4.4526 - acc: 0.0569 - val_loss: 4.4610 - val_acc: 0.1964
Epoch 2/15
352/352 [==============================] - 29s 82ms/step - loss: 3.7121 - acc: 0.2465 - val_loss: 3.2691 - val_acc: 0.3282
Epoch 3/15
352/352 [==============================] - 30s 85ms/step - loss: 2.7237 - acc: 0.3848 - val_loss: 2.3017 - val_acc: 0.4512
Epoch 4/15
352/352 [==============================] - 30s 85ms/step - loss: 2.0630 - acc: 0.4926 - val_loss: 1.9989 - val_acc: 0.5162
Epoch 5/15
352/352 [==============================] - 30s 84ms/step - loss: 1.6920 - acc: 0.5671 - val_loss: 1.8303 - val_acc: 0.5428
Epoch 6/15
352/352 [==============================] - 31s 88ms/step - loss: 1.4179 - acc: 0.6291 - val_loss: 1.7509 - val_acc: 0.5608
Epoch 7/15
352/352 [==============================] - 30s 85ms/step - loss: 1.1795 - acc: 0.6900 - val_loss: 1.7148 - val_acc: 0.5714
Epoch 8/15
352/352 [==============================] - 30s 85ms

Generate Test dataset

In [15]:
x_test = []
for i in test_dir:
    img = Image.open(i).convert("RGB")
    img = np.asarray(img)
    x_test.append(img)

x_test = np.asarray(x_test)

In [16]:
print(x_test.shape)

(5000, 64, 64, 3)


Prediction

In [17]:
import csv
from keras.preprocessing import image
final_predictions = [0] * 5000
filename = 'tiny-imagenet-100/submission_sample.csv'

with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    next(datareader)
    n = 0
    for row, i in datareader:
        img_path = 'tiny-imagenet-100/test/images/%s.JPEG' % row
        #print(img_path)
        img = image.load_img(img_path, target_size=(64, 64))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        #x = preprocess_input(x)
        preds = model.predict(x)
        final_predictions[n] = np.argmax(preds)
        #preds.argmax()
        #tf.math.argmax(predicted_batch, axis=-1)
        n = n + 1
print(final_predictions)


[9, 80, 59, 77, 80, 91, 79, 77, 16, 25, 26, 55, 85, 28, 48, 41, 50, 76, 53, 18, 94, 22, 65, 43, 59, 51, 46, 83, 12, 28, 82, 85, 43, 12, 95, 87, 8, 15, 28, 46, 83, 1, 2, 54, 0, 85, 11, 56, 28, 25, 40, 18, 67, 58, 44, 22, 46, 49, 11, 63, 5, 38, 30, 34, 84, 13, 42, 29, 68, 4, 87, 17, 10, 46, 94, 30, 86, 5, 1, 32, 54, 82, 33, 57, 62, 74, 38, 45, 51, 76, 59, 25, 34, 44, 39, 45, 5, 88, 29, 47, 93, 66, 78, 41, 62, 67, 4, 5, 13, 21, 18, 30, 1, 36, 82, 41, 6, 86, 74, 19, 76, 37, 5, 7, 55, 96, 71, 86, 60, 98, 39, 32, 7, 82, 59, 19, 12, 30, 81, 62, 32, 37, 49, 70, 42, 16, 66, 34, 57, 68, 13, 97, 31, 59, 10, 29, 8, 39, 40, 63, 91, 98, 93, 69, 49, 48, 98, 28, 81, 82, 15, 90, 42, 65, 35, 90, 1, 65, 58, 98, 30, 57, 79, 57, 81, 16, 61, 92, 35, 98, 33, 44, 49, 53, 66, 34, 35, 48, 51, 69, 30, 91, 73, 76, 20, 76, 62, 36, 83, 51, 46, 48, 87, 62, 90, 60, 12, 23, 43, 39, 28, 39, 49, 53, 13, 71, 23, 80, 34, 47, 14, 25, 60, 85, 3, 46, 50, 46, 31, 27, 85, 7, 12, 95, 0, 46, 3, 17, 19, 3, 77, 74, 24, 19, 86, 88,

Generate .csv file 

In [18]:
import pandas as pd
  
# reading the csv file
df = pd.read_csv("tiny-imagenet-100/submission_sample.csv")

for i in range(5000):
    df.loc[i, 'label'] = final_predictions[i]
    # writing into the file
    df.to_csv("submission_sample.csv", index=False)

Export the trained model

In [19]:
t = time.time()

export_path = "/content/saved_models/{}".format(int(t))
model.save(export_path)

export_path

INFO:tensorflow:Assets written to: /content/saved_models/1647408896/assets


'/content/saved_models/1647408896'

In [20]:
reloaded = tf.keras.models.load_model(export_path)

zip the saved model file and save it into google drive

In [21]:
!zip -r /content/saved_models.zip /content/saved_models

  adding: content/saved_models/ (stored 0%)
  adding: content/saved_models/1647408896/ (stored 0%)
  adding: content/saved_models/1647408896/assets/ (stored 0%)
  adding: content/saved_models/1647408896/saved_model.pb (deflated 92%)
  adding: content/saved_models/1647408896/keras_metadata.pb (deflated 95%)
  adding: content/saved_models/1647408896/variables/ (stored 0%)
  adding: content/saved_models/1647408896/variables/variables.index (deflated 82%)
  adding: content/saved_models/1647408896/variables/variables.data-00000-of-00001 (deflated 7%)


In [22]:
from google.colab import drive
drive.mount('/content/drive')
import shutil
shutil.copy("/content/saved_models.zip","/content/drive/MyDrive")

Mounted at /content/drive


'/content/drive/MyDrive/saved_models.zip'